In [1]:
!pip install -q flwr torch torchvision tensorboard
!pip install -U "flwr[simulation]"
!pip install timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.0/540.0 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 117.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install timm torchvision
!pip install wandb
!pip install torch_optimizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 6.3 MB/s eta 0:00:00


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR
from torch_optimizer import Ranger
from torchvision import transforms
from torchvision.datasets import CIFAR100
from torch.utils.data import DataLoader, random_split
import timm
from timm.loss import LabelSmoothingCrossEntropy
from timm.models.layers import DropPath
import data_preprocessing
from wandb_logger import WandBLogger
from torch.cuda.amp import autocast, GradScaler
import os
import random
import numpy as np
from datetime import datetime
import editing
from editing import *

/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [6]:
SEED = 42
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(SEED)

In [31]:
# THE WHOLE TRAINING SETTINGS ARE HERE!

# GENERAL
BATCH_SIZE = 256
EPOCHS = 15
VAL_SPLIT = 0.1
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BACKBONE_FREEZING = False

# LOSS
SMOOTHING=0.1
# OPTIMIZER
OPTIMIZER_NAME = "sadamw"
LR=0.00005
BETAS=(0.9, 0.999)
WEIGHT_DECAY=0.05
WARMUP_EPOCHS=5
NESTEROV=False
MOMENTUM=0.9
EPSILON = 1e-8
# SCHEDULER
SCHEDULER_NAME = "CosineAnnealingWarmRestarts"
START_FACTOR=1e-6 / 5e-5
T_MAX=EPOCHS - WARMUP_EPOCHS
# Early Stopping
PATIENCE = 6

#TALOS CONFIG
FINAL_SPARSITY = 0.6
NUM_BATCHES = 4
ROUNDS = 4
CALIBRATION_MODE = "most_sensitive" # least_sensitive or most_sensitive
MODE = "full" # full if want to include all layers, head for head only, pfededit for custom topk client selection
K = 3 #pfedit setting
FACTOR = 0.5 #probability of not doing pfededit

TALOS_CONFIG = {
    "final_sparsity": FINAL_SPARSITY,
    "num_batches": NUM_BATCHES,
    "rounds": ROUNDS,
    "calibration_mode": CALIBRATION_MODE,
    "mode": MODE,
    "k": K,
    "factor": FACTOR
}

In [32]:
!wandb login a8e9e7fc10d033043f95b8b53c354022103a54e7

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [33]:
pipeline = data_preprocessing.CIFAR100Pipeline(val_split=VAL_SPLIT, use_augment=True)
trainset, valset, testset = pipeline.run_pipeline()
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
valloader = DataLoader(valset, batch_size=BATCH_SIZE)
testloader = DataLoader(testset, batch_size=BATCH_SIZE)

In [34]:
# Create model
def create_dino_vit_s16_for_cifar100(freezing=BACKBONE_FREEZING):
    model = timm.create_model("vit_small_patch16_224_dino", pretrained=True, num_classes=0)

    # Replace the head with CIFAR-100 classification head
    model.head = nn.Linear(model.num_features, 100)

    if freezing:
      # Freeze all parameters except head
      for param in model.parameters():
          param.requires_grad = False

      # Unfreeze only the head
      for param in model.head.parameters():
          param.requires_grad = True
    else:

      for param in model.parameters():
          param.requires_grad = True

      for param in model.head.parameters():
          param.requires_grad = True

    return model

model = create_dino_vit_s16_for_cifar100(BACKBONE_FREEZING).to(DEVICE)


/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name vit_small_patch16_224_dino to current vit_small_patch16_224.dino.
  model = create_fn(


In [35]:
def build_pruner():
    pruner = TaLoSPruner(
        model = model,
        final_sparsity=FINAL_SPARSITY,
        device = DEVICE,
        num_batches=NUM_BATCHES,
        rounds=ROUNDS,
        mode=MODE
    )
    return pruner

pruner = build_pruner()
pruner.calibrate_masks(trainloader, strategy=CALIBRATION_MODE)

🟢 Pruning will be applied to the entire model.
🔎 Starting multi-round calibration for mode 'full'.
🌀 Calibration Round 1/4
📝 Calculating Fisher Information on 4 batches...
✅ Fisher Information Computation Completed.
🌀 Calibration Round 2/4
📝 Calculating Fisher Information on 4 batches...
✅ Fisher Information Computation Completed.
🌀 Calibration Round 3/4
📝 Calculating Fisher Information on 4 batches...
✅ Fisher Information Computation Completed.
🌀 Calibration Round 4/4
📝 Calculating Fisher Information on 4 batches...
✅ Fisher Information Computation Completed.
✅ Mask Calibration Completed!


In [36]:
def build_optimizer(optimizer_name, model, masks=None):
    if optimizer_name.lower() == "ssgd":
        config = {
            "lr": LR,
            "momentum": MOMENTUM,
            "weight_decay": WEIGHT_DECAY,
        }
        optimizer = editing.SparseSGDM(
            model.parameters(),
            lr=LR,
            momentum=MOMENTUM,
            weight_decay=WEIGHT_DECAY,
            masks=pruner.masks
        )
    elif optimizer_name.lower() == "sadamw":
        config = {
            "lr": LR,
            "betas": BETAS,
            "weight_decay": WEIGHT_DECAY,
            "eps": EPSILON

        }
        optimizer = editing.SparseAdamW(
            model.parameters(),
            lr=LR,
            betas=BETAS,
            weight_decay=WEIGHT_DECAY,
            eps=EPSILON,
            masks=pruner.masks
        )
    else:
        raise ValueError(f"Unsupported optimizer: {optimizer_name}")

    print(f"Optimizer '{optimizer_name}' initialized successfully.")
    return config, optimizer

def build_scheduler(scheduler_name, optimizer):
    if scheduler_name.lower() == "ranger + onecyclelr":
        config = {
            "max_lr": LR,
            "epochs": EPOCHS,
            "steps_per_epoch": len(trainloader),
            "pct_start": 0.3,
            "anneal_strategy": "cos",
            "div_factor": 25.0,
            "final_div_factor": 1e4
        }
        scheduler = OneCycleLR(
            optimizer,
            max_lr=LR,
            epochs=EPOCHS,
            steps_per_epoch=len(trainloader),
            pct_start=0.3,
            anneal_strategy="cos",
            div_factor=25.0,
            final_div_factor=1e4
        )
    elif scheduler_name.lower() == "cosineannealinglr + warmup":
        config = {
            "warmup_epochs": WARMUP_EPOCHS,
            "start_factor": START_FACTOR,
            "t_max": T_MAX,
            "milestones": [WARMUP_EPOCHS]
        }
        scheduler = optim.lr_scheduler.SequentialLR(
            optimizer,
            schedulers=[
                optim.lr_scheduler.LinearLR(optimizer, start_factor=START_FACTOR, total_iters=WARMUP_EPOCHS),
                optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_MAX)
            ],
            milestones=[WARMUP_EPOCHS]
        )
    elif scheduler_name.lower() == "cosineannealingwarmrestarts":
        config = {
            "T_0": 10,
            "T_mult": 2
        }
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer,
            T_0=10,
            T_mult=2
        )
    else:
        raise ValueError(f"Unsupported scheduler: {scheduler_name}")

    print(f"Scheduler '{scheduler_name}' initialized successfully.")
    return config, scheduler



In [37]:
OPTIMIZER_CONFIG, optimizer = build_optimizer(OPTIMIZER_NAME, model)
SCHEDULER_CONFIG, scheduler = build_scheduler(SCHEDULER_NAME, optimizer)

🔍 Mapping parameters to their masks...
✅ Mapped 150 parameters to masks.
Optimizer 'sadamw' initialized successfully.
Scheduler 'CosineAnnealingWarmRestarts' initialized successfully.


In [39]:
print(next(model.parameters()).device)
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable:,} / {total:,}")

torch.backends.cudnn.benchmark = True

cuda:0
Trainable params: 21,704,164 / 21,704,164


In [40]:
run_name = f"CENTRALIZED_TALOS_{datetime.now().strftime('%Y%m%d-%H%M%S')}"

logger = WandBLogger(
    project_name="federated-learning-project",
    run_name=run_name,
    config={
        # Training Setup
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "device": DEVICE,
        "backbone_freezing": BACKBONE_FREEZING,

        # Loss Configuration
        "label_smoothing": SMOOTHING,

        # Optimizer Configuration
        "optimizer": OPTIMIZER_NAME,
        "optimizer_config": OPTIMIZER_CONFIG,

        # Scheduler Configuration
        "scheduler": SCHEDULER_NAME,
        "scheduler_config": SCHEDULER_CONFIG,

        # Talos Configuration
        "talos_config": TALOS_CONFIG,

        # Early Stopping
        "patience": PATIENCE
    }
)


In [ ]:
# Loss with label smoothing
criterion = LabelSmoothingCrossEntropy(smoothing=SMOOTHING)

scaler = GradScaler()

best_val_acc = 0.0
epochs_no_improve = 0

# Training loop
for epoch in range(EPOCHS):
    model.train()
    correct, total, train_loss = 0, 0, 0.0

    for x, y in trainloader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()

        with autocast():
            outputs = model(x)
            loss = criterion(outputs, y)

        scaler.scale(loss).backward()
        scaler.step(optimizer) #Optimizer applies the mask!!!
        scaler.update()

        # scheduler.step()

        train_loss += loss.item() * y.size(0)
        _, pred = torch.max(outputs, 1)
        correct += (pred == y).sum().item()
        total += y.size(0)

    scheduler.step()

    train_acc = correct / total
    train_loss /= total

    logger.log_metrics({
      "train_loss": train_loss,
      "train_acc": train_acc,
      "learning_rate": scheduler.get_last_lr()[0]
    }, step=epoch)


    # Validation
    model.eval()
    correct, total, val_loss = 0, 0, 0.0
    with torch.no_grad():
        for x, y in valloader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            outputs = model(x)
            loss = criterion(outputs, y)

            val_loss += loss.item() * y.size(0)
            _, pred = torch.max(outputs, 1)
            correct += (pred == y).sum().item()
            total += y.size(0)

    val_acc = correct / total
    val_loss /= total
    logger.log_metrics({
      "val_loss": val_loss,
      "val_acc": val_acc
    }, step=epoch)

    print(f"Epoch {epoch+1:02d}/{EPOCHS} — Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    # Early stopping logic
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        epochs_no_improve = 0
        best_model_state = model.state_dict()  # save best model
        logger.log_model(model, path="best_model.pth")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

model.load_state_dict(best_model_state)

<ipython-input-41-556336675>:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-41-556336675>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


In [30]:
#  test
model.eval()
correct, total, test_loss = 0, 0, 0.0
with torch.no_grad():
    for x, y in testloader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        outputs = model(x)
        loss = criterion(outputs, y)

        test_loss += loss.item() * y.size(0)
        _, pred = torch.max(outputs, 1)
        correct += (pred == y).sum().item()
        total += y.size(0)

test_acc = correct / total
test_loss /= total

logger.log_metrics({
    "test_loss": test_loss,
    "test_acc": test_acc
})

logger.finish()

print(f"\n Final Test Accuracy: {test_acc:.4f} | Test Loss: {test_loss:.4f}")


learning_rate,█▇▇▆▄▃▂▂▁████▇▇
test_acc,▁
test_loss,▁
train_acc,▁▅▇████████████
train_loss,█▃▂▂▁▁▁▁▁▁▁▂▁▁▁
val_acc,▁▅▇▇▇█████▆▇███
val_loss,█▄▂▂▂▂▂▂▂▂▃▂▁▁▁
learning_rate,4e-05
test_acc,0.8639
test_loss,1.26236
train_acc,0.99951



 Final Test Accuracy: 0.8639 | Test Loss: 1.2624
